In [2]:
import ast
import astor
class InputVisitorDetector(ast.NodeVisitor):
    def __init__(self):
        self.user_inputs_all = {}

    def visit_Assign(self, node):
        for child in ast.walk(node.value):
            if isinstance(child, ast.Call):
                if isinstance(child.func, ast.Name):
                    if child.func.id == 'input':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                print(target.id)
                                if(str(target.id)  not in self.user_inputs_all):
                                   self.user_inputs_all[target.id]="str"
                    elif child.func.id == 'int':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs_all[target.id] ="int"
                                break
                    elif child.func.id == 'float':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs_all[target.id] = "float"
                                break
        self.generic_visit(node)
        
class InputVisitor(ast.NodeTransformer):
    def __init__(self):
        self.user_inputs = {}
        #self.param_names = ["param" + str(i) for i in range(1, 11)] # Değişken isimleri
        self.count=0

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "input":
            # input fonksiyonunu değiştir
            self.count+=1
            new_name = "param"+str(self.count)
            self.user_inputs[new_name] = str
            new_node=ast.Name(id=new_name, ctx=ast.Load())
            return ast.copy_location(new_node, node)
        else:
            return self.generic_visit(node)
    
import ast

class RemoveTypeConversions(ast.NodeTransformer):
    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id in ("int", "float", "str"):
            # int(param), float(param), veya str(param) gibi ifadeleri kaldırma
            if isinstance(node.args[0], ast.Name):
                param_name = node.args[0].id
                return ast.copy_location(ast.Name(id=param_name, ctx=ast.Load()), node)
        return self.generic_visit(node)

def type_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitorDetector()
    visitor.visit(tree)
    return visitor.user_inputs_all

def detect_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitor()
    visitor.visit(tree)
    new_tree1=ast.fix_missing_locations(tree)
    transformer = RemoveTypeConversions()
    new_tree = transformer.visit(new_tree1)
    new_tree = ast.fix_missing_locations(new_tree)
    my_real_inputs=list(type_user_inputs(code_file).values())
    print(my_real_inputs)
    function_node = ast.FunctionDef(
    name=code_file[:-3],
    args=ast.arguments(
        args=[ast.arg(arg=list(visitor.user_inputs.keys())[i], annotation=None) for i in range(len(visitor.user_inputs.keys()))],
        vararg=None,
        kwonlyargs=[],
        kw_defaults=[],
        kwarg=None,
        defaults=[]
    ),
    body=new_tree.body,
    decorator_list=[],
    returns=None
)
    with open("final_file_"+code_file, "w") as f:
        f.write(astor.to_source(function_node))
    return visitor.user_inputs


In [21]:
detect_user_inputs("code_2.py")

knight_hor
knight_column
bishop_hor
bishop_column
['str', 'int', 'str', 'int']


{'param1': str, 'param2': str, 'param3': str, 'param4': str}

In [3]:
detect_user_inputs("code_3.py")

database
movement_
budget
purchases
['str', 'str', 'float', 'str', 'float']


{'param1': str, 'param2': str, 'param3': str, 'param4': str}

In [3]:
import ast
import astor

def convert_code_to_function(code):
    tree = ast.parse(code)
    
    new_function = ast.FunctionDef(
        name='new_function',  #function name
        args=ast.arguments(args=[], defaults=[], kwonlyargs=[], kw_defaults=[]),  # Arguments
        body=tree.body,  # Code inside 
        decorator_list=[]  
    )
    tree.body = [new_function]
    return astor.to_source(new_function)


In [ ]:
# Example Python code
example_code = '''
img_width = int(input("Please enter the image width in pixels: "))
img_height = int(input("Please enter the image height in pixels: "))

total_bytes = img_width * img_height * color_chnl_byte
total_bits = total_bytes * 8

modem_transmission_delay = total_bits / modem_bandwidth
ethernet_transmission_delay = total_bits / ethernet_bandwidth
for i in range(1,4):
    x=input ("Enter the code file name: ")
modem_delay_hours = modem_transmission_delay // seconds_in_hours
modem_delay_minutes = (modem_transmission_delay - modem_delay_hours*seconds_in_hours) // 60
model_delay_seconds = modem_transmission_delay % 60
'''

function_code = convert_code_to_function(example_code)  # Process the code

print(function_code)  # The generated code of the new function

In [1]:
import re

def extract_function_names(file_path):
    with open(file_path, 'r') as file:
        code = file.read()
        function_pattern = r"def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\("
        match = re.search(function_pattern, code)
        if match:
            return match.group(1)
        else:
            return None

file_path = 'codedef.txt'  # Replace this with your file path
function_name = extract_function_names(file_path)
print(function_name)


power


In [2]:
import ast

def check_for_functions(code):
    try:
        parsed = ast.parse(code)
        for node in ast.walk(parsed):
            if isinstance(node, ast.FunctionDef):
                return True  # Eğer fonksiyon bulursa True döner
        return False  # Eğer fonksiyon bulamazsa False döner
    except SyntaxError:
        return False  # Geçersiz sözdizimi hatası varsa da False döner

# Örnek bir kod
sample_code = """

    print("Hello, world!")

x = 5
"""

result = check_for_functions(sample_code)
print(result)


Bu kod içinde fonksiyon var mı? True
